In [2]:
import pandas as pd
import numpy as np
from sklearn import *
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("/data/credit-default.csv")
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
checking_balance        1000 non-null object
months_loan_duration    1000 non-null int64
credit_history          1000 non-null object
purpose                 1000 non-null object
amount                  1000 non-null int64
savings_balance         1000 non-null object
employment_length       1000 non-null object
installment_rate        1000 non-null int64
personal_status         1000 non-null object
other_debtors           1000 non-null object
residence_history       1000 non-null int64
property                1000 non-null object
age                     1000 non-null int64
installment_plan        1000 non-null object
housing                 1000 non-null object
existing_credits        1000 non-null int64
default                 1000 non-null int64
dependents              1000 non-null int64
telephone               1000 non-null object
foreign_worker          1000 non-null object
jo

In [5]:
df.default.value_counts()/len(df)

1    0.7
2    0.3
Name: default, dtype: float64

In [15]:
target = "default"
y = np.where(df[target] == 2, 1, 0) #outcome variable
X = df.copy() #feature matrix
del X[target]
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,
                        test_size = 0.3, random_state = 1234)
X_train.shape, X_test.shape

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", tree.DecisionTreeClassifier(max_depth=4, criterion="entropy", random_state=1))
])

pipe.fit(X_train, y_train)
print("training: ", pipe.score(X_train, y_train),
"\ntesting: ", pipe.score(X_test, y_test))

training:  0.7528571428571429 
testing:  0.69


In [14]:
from sklearn.tree import export_graphviz

est = pipe.steps[-1][-1]
export_graphviz(est, out_file = "tree.dot", feature_names = X.columns, filled=True)
!dot -Tpng tree.dot -o tree.png


In [25]:
est = pipe.steps[-1][-1]
pd.DataFrame({"feature": X.columns, "importance": est.feature_importances_}).sort_values("importance", ascending = False)

,feature,importance
9,checking_balance_unknown,0.314839
1,amount,0.160376
0,months_loan_duration,0.107381
39,installment_plan_none,0.061100
4,age,0.047717
38,property_unknown/none,0.044055
28,employment_length_4 - 7 yrs,0.036642
12,credit_history_fully repaid this bank,0.028131
35,other_debtors_none,0.020515
2,installment_rate,0.018847


In [24]:
target = "default"
y = np.where(df[target] == 2, 1, 0) #outcome variable
X = df.copy() #feature matrix
del X[target]
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y,
                        test_size = 0.3, random_state = 1234)
X_train.shape, X_test.shape

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", ensemble.RandomForestClassifier(max_features=0.5, n_estimators= 10, 
        max_depth=4, criterion="entropy", random_state=1))
])

pipe.fit(X_train, y_train)
print("training: ", pipe.score(X_train, y_train),
"\ntesting: ", pipe.score(X_test, y_test))

training:  0.7785714285714286 
testing:  0.7133333333333334


In [27]:
est = pipe.steps[-1][-1]
pd.DataFrame({"feature": X.columns, 
              "importance": est.feature_importances_})\
.sort_values("importance", ascending = False)

,feature,importance
9,checking_balance_unknown,0.314839
1,amount,0.160376
0,months_loan_duration,0.107381
39,installment_plan_none,0.061100
4,age,0.047717
38,property_unknown/none,0.044055
28,employment_length_4 - 7 yrs,0.036642
12,credit_history_fully repaid this bank,0.028131
35,other_debtors_none,0.020515
2,installment_rate,0.018847


In [28]:
est.estimators_

[DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
             max_features=0.5, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False,
             random_state=1791095845, splitter='best'),
 DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
             max_features=0.5, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False,
             random_state=2135392491, splitter='best'),
 DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
             max_features=0.5, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_frac